# Google form analysis

Analysis of results extracted from Google forms in csv format.

# Table of Contents

1.  [Google form loading](#gformload)

    1.  [Selection of a question](#selquest)

    2.  [Selection of a user's answers](#selusans)

2.  [correct answers list](#corranslist)

3.  [checking answers](#checkans)

4.  [checkpoint / question association](#checkquesmatch)

5.  [comparison of checkpoints completion and answers](#compcheckans)

6.  [Functions](#functions)

## Preparation

In [183]:
%run Utilities/Preparation.ipynb
df152 = pd.read_csv("../data/2017-09-05.1.52.csv")
df152 = df152.rename(index=str, columns={'playerId': 'sessionId'})

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (24,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [184]:
# Localplayerguids of users who answered the questionnaire (see below).
# French
#localplayerguid = '"a4d4b030-9117-4331-ba48-90dc05a7e65a"'
#localplayerguid = '"d6826fd9-a6fc-4046-b974-68e50576183f"'
#localplayerguid = '"deb089c0-9be3-4b75-9b27-28963c77b10c"'
#localplayerguid = '"75e264d6-af94-4975-bb18-50cac09894c4"'
#localplayerguid = '"3d733347-0313-441a-b77c-3e4046042a53"'
# English
localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'
#localplayerguid = '"7037c5b2-c286-498e-9784-9a061c778609"'
#localplayerguid = '"5c4939b5-425b-4d19-b5d2-0384a515539e"'
#localplayerguid = '"7825d421-d668-4481-898a-46b51efe40f0"'
#localplayerguid = '"acb9c989-b4a6-4c4d-81cc-6b5783ec71d8"'
#localplayerguid = devPCID5

## 1. Google form loading
<a id=gformload />

In [185]:
#formname = '2017-09-12-gform-fr.csv'
formname = '2017-09-12-gform-en.csv'

#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = 34

gform = pd.read_csv("../data/Google forms/" + formname)

In [186]:
transposed = gform.T
#answers = transposed[transposed[]]
transposed

,0,1,2,3
Timestamp,2017/08/18 4:22:26 PM GMT+2,2017/09/06 2:28:03 PM GMT+2,2017/09/12 10:02:06 AM GMT+2,2017/09/13 3:11:34 AM GMT+2
Are you interested in video games?,Extremely,Moderately,A lot,I don't know
Do you play video games?,Extremely,Moderately,Moderately,I don't know
How old are you?,23,28,20,0
What is your gender?,Female,Other,Female,Other
How long have you studied biology?,Until bachelor's degree,Until the end of high school,Until bachelor's degree,I don't know
Are you interested in biology?,Extremely,Moderately,Moderately,I don't know
Have you ever heard about synthetic biology or BioBricks?,Yes,Yes,No,No
Have you ever played an older version of Hero.Coli before?,No,No,No,No
Have you played the current version of Hero.Coli?,Yes,Yes,Yes,Yes


In [187]:
type(gform)

pandas.core.frame.DataFrame

### 1.1 Selection of a question
<a id=selquest />

In [188]:
# Using the whole question:
gform[localplayerguidkey]

0    8d352896-a3f1-471c-8439-0f426df901c1
1    7037c5b2-c286-498e-9784-9a061c778609
2    5c4939b5-425b-4d19-b5d2-0384a515539e
3    7825d421-d668-4481-898a-46b51efe40f0
Name: Do not edit -  pre-filled anonymous ID, dtype: object

In [189]:
# Using the index of the question:
gform.iloc[:, localplayerguidindex]

0    8d352896-a3f1-471c-8439-0f426df901c1
1    7037c5b2-c286-498e-9784-9a061c778609
2    5c4939b5-425b-4d19-b5d2-0384a515539e
3    7825d421-d668-4481-898a-46b51efe40f0
Name: Do not edit -  pre-filled anonymous ID, dtype: object

### 1.2 Selection of a user's answers
<a id=selusans />

In [190]:
# Localplayerguids stored in gform have different format
answers = gform[gform[localplayerguidkey]==localplayerguid.replace('"','')]
columnAnswers = answers.T
columnAnswers.columns = ['answers']
columnAnswers

,answers
Timestamp,2017/08/18 4:22:26 PM GMT+2
Are you interested in video games?,Extremely
Do you play video games?,Extremely
How old are you?,23
What is your gender?,Female
How long have you studied biology?,Until bachelor's degree
Are you interested in biology?,Extremely
Have you ever heard about synthetic biology or BioBricks?,Yes
Have you ever played an older version of Hero.Coli before?,No
Have you played the current version of Hero.Coli?,Yes


In [191]:
# Selection of a specific answer
answers.iloc[:,localplayerguidindex]

0    8d352896-a3f1-471c-8439-0f426df901c1
Name: Do not edit -  pre-filled anonymous ID, dtype: object

In [192]:
answers.iloc[:,localplayerguidindex].iloc[0]

'8d352896-a3f1-471c-8439-0f426df901c1'

In [193]:
type(answers.iloc[0,:])

pandas.core.series.Series

In [194]:
answers.iloc[0,:].values

array(['2017/08/18 4:22:26 PM GMT+2', 'Extremely', 'Extremely', 23,
       'Female', "Until bachelor's degree", 'Extremely', 'Yes', 'No',
       'Yes', 'E. Coli', 'Edit the DNA of the bacterium', 'DNA sequences',
       'Flagella', 'Ampicillin', 'Terminator', 'Promoter', 'Fluorescence',
       'RBS', 'Option 2',
       'Under blue light, when the GFP device is equipped',
       'Flagella quickly disappear one by one',
       'A plasmid - it makes it possible to equip an additional device',
       "I don't know", "I don't know",
       'It generates antibiotic resistance',
       'It generates green fluorescence in presence of l-arabinose',
       'It generates green fluorescence',
       'It makes it possible to move faster',
       'It generates antibiotic resistance',
       'It generates green fluorescence in presence of l-arabinose',
       "I don't know",
       'After being induced, it would produce more and more l-arabinose, because it would induce itself',
       'Your game is 

## 2. correct answers list
<a id=corranslist />

In [195]:
%run "Static data/Google form correct answers.ipynb"
#correctAnswers

## 3. checking answers
<a id=checkans />

#### Question that has a correct answer:

In [196]:
questionIndex = 15
answers.iloc[:,questionIndex].iloc[0].startswith(correctAnswers.iloc[questionIndex].iloc[0])

True

#### Question that has no correct answer:

In [197]:
questionIndex = 0
answers.iloc[:,questionIndex].iloc[0].startswith(correctAnswers.iloc[questionIndex].iloc[0])

True

#### Batch check:

In [198]:
columnAnswers.values[2,0]

'Extremely'

In [199]:
columnAnswers['answers'][2]

'Extremely'

In [200]:
correctAnswers['correctAnswer']

0                                                      
1                                                      
2                                                      
3                                                      
4                                                      
5                                                      
6                                                      
7                                                      
8                                                      
9                                                      
10                                              E. Coli
11                        Edit the DNA of the bacterium
12                                        DNA sequences
13                                             Flagella
14                                           Ampicillin
15                                           Terminator
16                                             Promoter
17                                      Coding S

In [201]:
#print(correctAnswers)
#print(columnAnswers)
#type(correctAnswers)
#type(columnAnswers)

#columnAnswers['binaryCorrect'] = columnAnswers.values==correctAnswers.values
columnAnswers['correct'] = columnAnswers['answers']

for index in range(0, 10):
    columnAnswers['correct'][index] = 'NaN'
for index in range(10, len(columnAnswers.values)-1):
    columnAnswers['correct'][index] = str(columnAnswers['answers'][index]).startswith(str(correctAnswers['correctAnswer'][index]))
columnAnswers['correct'][len(columnAnswers.values)-1] = 'NaN'
columnAnswers

,answers,correct
Timestamp,2017/08/18 4:22:26 PM GMT+2,NaN
Are you interested in video games?,Extremely,NaN
Do you play video games?,Extremely,NaN
How old are you?,23,NaN
What is your gender?,Female,NaN
How long have you studied biology?,Until bachelor's degree,NaN
Are you interested in biology?,Extremely,NaN
Have you ever heard about synthetic biology or BioBricks?,Yes,NaN
Have you ever played an older version of Hero.Coli before?,No,NaN
Have you played the current version of Hero.Coli?,Yes,NaN


In [202]:
score = (columnAnswers['correct']).value_counts()[True]
score

19

## 4. checkpoint / question association
<a id=checkquesmatch />

In [203]:
# done in already imported 'correct answers.ipynb'

## 5. comparison of checkpoints completion and answers
<a id=compcheckans />

Theoretically, they should match. Whoever understood an item should beat the matching challenge. The discrepancies are due to game design or level design.

In [204]:
#questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(35))
questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
questionnaireValidatedCheckpointsPerQuestion.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64

In [205]:
checkpointQuestionMatching['checkpoint'][19]

'tutorial1.Checkpoint00'

In [206]:
for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
    if columnAnswers['correct'][index]==True:
        questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
    else:
        questionnaireValidatedCheckpointsPerQuestion[index] = ''
questionnaireValidatedCheckpointsPerQuestion

0                           
1                           
2                           
3                           
4                           
5                           
6                           
7                           
8                           
9                           
10    tutorial1.Checkpoint00
11    tutorial1.Checkpoint00
12    tutorial1.Checkpoint00
13    tutorial1.Checkpoint00
14    tutorial1.Checkpoint02
15    tutorial1.Checkpoint01
16    tutorial1.Checkpoint05
17                          
18    tutorial1.Checkpoint05
19                          
20    tutorial1.Checkpoint02
21    tutorial1.Checkpoint00
22    tutorial1.Checkpoint05
23                          
24                          
25    tutorial1.Checkpoint13
26    tutorial1.Checkpoint05
27    tutorial1.Checkpoint02
28    tutorial1.Checkpoint02
29    tutorial1.Checkpoint13
30    tutorial1.Checkpoint05
31                          
32    tutorial1.Checkpoint05
33                          
34            

In [207]:
questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
questionnaireValidatedCheckpoints

0    tutorial1.Checkpoint00
1    tutorial1.Checkpoint01
2    tutorial1.Checkpoint02
3    tutorial1.Checkpoint05
4    tutorial1.Checkpoint13
dtype: object

In [208]:
#for index in range(len(questionnaireValidatedCheckpoints), 0):
#for index in range(0, len(questionnaireValidatedCheckpoints)):
#    if questionnaireValidatedCheckpoints
#    print(index)
    
#questionnaireValidatedCheckpoints
#type(questionnaireValidatedCheckpoints)

# Functions
<a id=functions />

In [228]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    columnAnswers = answers.T
    columnAnswers.columns = ['answers']
    return columnAnswers

def getCorrections( userId):
    columnAnswers = getAnswers( userId )
    
    columnAnswers['correct'] = columnAnswers['answers']

    for index in range(0, 10):
        columnAnswers['correct'][index] = 'NaN'
    for index in range(10, len(columnAnswers.values)-1):
        columnAnswers['correct'][index] = str(columnAnswers['answers'][index]).startswith(str(correctAnswers['correctAnswer'][index]))
    columnAnswers['correct'][len(columnAnswers.values)-1] = 'NaN'
    return columnAnswers

#function that returns the score from user id
def getScore( userId ):
    if hasAnswered( userId ):
        columnAnswers = getCorrections( userId)
        score = (columnAnswers['correct']).value_counts()[True]
        return score
    else:
        return np.NaN

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)



#### test

In [210]:
getAnswers(localplayerguid).head(2)

,answers
Timestamp,2017/08/18 4:22:26 PM GMT+2
Are you interested in video games?,Extremely


In [211]:
getCorrections(localplayerguid).head(2)

,answers,correct
Timestamp,2017/08/18 4:22:26 PM GMT+2,NaN
Are you interested in video games?,Extremely,NaN


In [212]:
getScore(localplayerguid)

19

In [213]:
getValidatedCheckpoints(localplayerguid)

0    tutorial1.Checkpoint00
1    tutorial1.Checkpoint01
2    tutorial1.Checkpoint02
3    tutorial1.Checkpoint05
4    tutorial1.Checkpoint13
dtype: object

In [214]:
getNonValidatedCheckpoints(localplayerguid)

0    
dtype: object

In [215]:
#localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'
#localplayerguid = '"7037c5b2-c286-498e-9784-9a061c778609"'
#localplayerguid = '"5c4939b5-425b-4d19-b5d2-0384a515539e"'
#localplayerguid = '"7825d421-d668-4481-898a-46b51efe40f0"'
#localplayerguid = '"acb9c989-b4a6-4c4d-81cc-6b5783ec71d8"'
userIds = [
'"8d352896-a3f1-471c-8439-0f426df901c1"'
,'"7037c5b2-c286-498e-9784-9a061c778609"'
,'"5c4939b5-425b-4d19-b5d2-0384a515539e"'
,'"7825d421-d668-4481-898a-46b51efe40f0"'
#,'"acb9c989-b4a6-4c4d-81cc-6b5783ec71d8"'
]
for id in userIds:
    print("===========================================")
    print(getAnswers(id).head(2))
    print("-------------------------------------------")
    print(getCorrections(id).head(2))
    print("-------------------------------------------")
    print(getScore(id))
    print("-------------------------------------------")
    print(getValidatedCheckpoints(id))
    print("-------------------------------------------")
    print(getNonValidatedCheckpoints(id))
    print("===========================================")

                                                        answers
Timestamp                           2017/08/18 4:22:26 PM GMT+2
Are you interested in video games?                    Extremely
-------------------------------------------
                                                        answers correct
Timestamp                           2017/08/18 4:22:26 PM GMT+2     NaN
Are you interested in video games?                    Extremely     NaN
-------------------------------------------
19
-------------------------------------------
0    tutorial1.Checkpoint00
1    tutorial1.Checkpoint01
2    tutorial1.Checkpoint02
3    tutorial1.Checkpoint05
4    tutorial1.Checkpoint13
dtype: object
-------------------------------------------
0    
dtype: object
                                                        answers
Timestamp                           2017/09/06 2:28:03 PM GMT+2
Are you interested in video games?                   Moderately
-------------------------------------------
   

In [216]:
#test
testSeries = pd.Series( 
    [
            '', # 7
            '', # 8
            '', # 9
            '', # 10

            'tutorial1.Checkpoint00', # 11
            'tutorial1.Checkpoint00', # 12
            'tutorial1.Checkpoint00', # 13
            'tutorial1.Checkpoint00', # 14
            'tutorial1.Checkpoint02', # 15
            'tutorial1.Checkpoint01', # 16
            'tutorial1.Checkpoint05'
        ]
    )
getNonValidated(testSeries)

0    tutorial1.Checkpoint13
dtype: object

In [217]:
gform[localplayerguidkey]
hasAnswered( '"8d352896-a3f1-471c-8439-0f426df901c1"' )
'8d352896-a3f1-471c-8439-0f426df901c1' in gform[localplayerguidkey].values

True